Author: Lai Khee Jiunn
  
  

Source: 

    [1] http://www.bursamalaysia.com/market/listed-companies/list-of-companies/main-market/
    [2] https://klse.i3investor.com/
    [3] https://www.klsescreener.com/
  
  

Reference:

    [1] K. M. Ho, "How to make money from your stock investment even in a falling market", 3rd ed., Malaysia, Kanyin Publications Sdn. Bhd., 2017.

Note: The entire process in this jupyter notebook may take up to 2.5 hours depending on your internet speed

In [1]:
import numpy as np
import pandas as pd
import re, time, os
from datetime import datetime, date
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
# from time import time
from tqdm import tqdm
from glob import glob
chromedriver = 'D://Workspace/selenium_driver/bin/chromedriver'
out_dir = 'result_2020-04-04/'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [2]:
stock_code = pd.read_csv(out_dir+'stock_code_table.csv')
data1 = pd.read_csv(out_dir+'klse_screener_52week_price.csv')
data2 = pd.read_csv(out_dir+'klse_morningstar_merge_split.csv', parse_dates=['split_date'])
data2['code'] = data2['code'].astype(str).apply(lambda x: x.zfill(4))
# data3_1 = pd.read_csv(out_dir+'klse_investor_10yr_price.csv')
# data3_1['code'] = data3_1['code'].astype(str).apply(lambda x: x.zfill(4))
# data3_2 = pd.read_csv(out_dir+'klse_morningstar_10yr.csv')
# data3_2['code'] = data3_2['code'].astype(str).apply(lambda x: x.zfill(4))
data3 = pd.read_csv(out_dir+'klse_10yr_data.csv')
data3['code'] = data3['code'].astype(str).apply(lambda x: x.zfill(4))
# data4 = pd.read_csv(out_dir+'klse_stock_stage_4.csv')
# data4['code'] = data4['code'].astype(str).apply(lambda x: x.zfill(4))
# data5 = pd.read_csv(out_dir+'klse_stock_intrinsic.csv')
# data5['code'] = data5['code'].astype(str).apply(lambda x: x.zfill(4))
# data6 = pd.read_csv(out_dir+'klse_stock_dividend_yield.csv')
# data6['code'] = data6['code'].astype(str).apply(lambda x: x.zfill(4))

### Get all listed company names and stock codes from Bursa Malaysia

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("--incognito")
driver = webdriver.Chrome(chromedriver, options=options)
driver.switch_to.window(driver.current_window_handle)
driver.maximize_window()
driver.get('http://www.bursamalaysia.com/market/listed-companies/list-of-companies/main-market/')
driver.find_element_by_class_name('cc-btn').click()
time.sleep(2)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight + 200);")
time.sleep(2)
driver.find_element_by_class_name('custom-select').click()
driver.find_element_by_class_name('custom-select').find_elements_by_tag_name('option')[-1].click()
time.sleep(2)
x = driver.find_elements_by_class_name('listed-company-table')[-1]\
          .find_element_by_tag_name('tbody')\
          .find_elements_by_class_name('company-announcement-link')

code, name = [],[]
for i in tqdm(x):
    code.append(i.get_attribute('href').split('=')[-1])
    name.append(i.get_attribute('innerText').strip())
driver.quit()

100%|████████████████████████████████████████████████████████████████████████████| 789/789 [00:06<00:00, 114.19it/s]


In [3]:
stock_code = pd.DataFrame({'code':code,'name':name})
idx1 = stock_code[stock_code.name == 'KLCC REAL ESTATE INVESTMENT TRUST'].index
idx2 = stock_code[stock_code.name == 'KLCC PROPERTY HOLDINGS BERHAD'].index
stock_code.loc[idx1, 'code'] = '5235SS'
stock_code.drop(index=idx2,inplace=True)
stock_code.reset_index(drop=True)
print(stock_code.shape)
stock_code.to_csv(out_dir+'stock_code_table.csv', index=False)
stock_code.head(1)

(789, 2)


,code,name
0,5250,7-ELEVEN MALAYSIA HOLDINGS BERHAD


### Stage 1: Get 52w high and low price

In [5]:
def stage_1(stock_code):
    # Get stock prices in 52 weeks range from KLSE Screener, which will be used in Stage 4
    start_time = time.time()
    options = webdriver.ChromeOptions()
    options.add_argument("--incognito")
    driver = webdriver.Chrome(chromedriver, options=options)
    driver.switch_to.window(driver.current_window_handle)
    driver.maximize_window()
    error1, result1 = [],[]
    for i in tqdm(stock_code.code):
        try:
            driver.get('https://www.klsescreener.com/v2/stocks/view/'+i)
            try:
                cat= driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/span[2]')\
                           .get_attribute('innerText')\
                           .split(' - ')[0]
            except:
                cat= 'UNKNOWN'
            price  = driver.find_element_by_id('price')\
                           .get_attribute('innerText')
            week52 = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/div/table[1]/tbody/tr[7]/td[2]')\
                           .get_attribute('innerText')\
                           .split(' - ')
            roe    = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/div/table[1]/tbody/tr[8]/td[2]')\
                           .get_attribute('innerText')\
                           .replace(',','')
            pe     = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/div/table[1]/tbody/tr[9]/td[2]')\
                           .get_attribute('innerText')
            eps    = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/div/table[1]/tbody/tr[10]/td[2]')\
                           .get_attribute('innerText')
            ey     = np.around(float(eps)/float(price), 2)
            dps    = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/div/table[1]/tbody/tr[11]/td[2]')\
                           .get_attribute('innerText')
            dy     = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/div/table[1]/tbody/tr[12]/td[2]')\
                           .get_attribute('innerText')\
                           .replace('%','')
            nta    = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/div/table[1]/tbody/tr[14]/td[2]')\
                           .get_attribute('innerText')
#             cap    = driver.find_element_by_xpath('//*[@id="page"]/div[2]/div[1]/div[2]/div[1]/div/table[1]/tbody/tr[18]/td[2]')\
#                            .get_attribute('innerText')

            if len(roe)==0:
                roe = 0.0

            result1.append([str(i), str(cat), float(price), float(nta), float(week52[0]), float(week52[-1]),
                            float(roe), float(pe), float(eps), ey, float(dps), float(dy)])
        except:
            error1.append(str(i))
    driver.quit()
    print('Time taken: ',time.time()-start_time)
    data1 = pd.DataFrame(result1, columns=['code','category','Price','NTA','52w low','52w high',
                                           'ROE','P/E','EPS','EY','DPS','DY'])
    return data1, error1
data1, error1 = stage_1(stock_code)
print(data1.shape)
data1.to_csv(out_dir+'klse_screener_52week_price.csv', index=False)
display(data1.head(1))
assert(len(error1) == 0)

100%|█████████████████████████████████████████████████████████████████████████████| 789/789 [27:10<00:00,  2.07s/it]


Time taken:  1641.7881543636322
(786, 13)


,code,category,price,NTA,52w low,52w high,market cap,ROE,P/E,EPS,EY,DPS,DY
0,5250,Consumer Products & Services,1.36,0.0893,1.21,1.56,"1,677.4M",49.05,31.0,4.38,3.22,2.4,1.76


100%|█████████████████████████████████████████████████████████████████████████████████| 3/3 [00:07<00:00,  2.37s/it]


Time taken:  18.298102378845215
(3, 13)


,code,category,price,NTA,52w low,52w high,market cap,ROE,P/E,EPS,EY,DPS,DY
0,5295,UNKNOWN,0.32,0.1382,0.242,0.716,225.9M,32.42,7.0,4.48,14.0,1.0,3.13


### Stage 2: Get Share Merges & Splits

In [3]:
# Get stock splits - number of merges and splits
start_time = time.time()
options = webdriver.ChromeOptions()
options.add_argument("--incognito")
driver = webdriver.Chrome(chromedriver, options=options)
driver.switch_to.window(driver.current_window_handle)
driver.maximize_window()
error2, result2 = [],[]
for i in tqdm(stock_code.code):
    try:
        driver.get('http://performance.morningstar.com/stock/performance-return.action?p=dividend_split_page&t='+str(i)+'&region=mys&culture=en-US')
#         splits = driver.find_element_by_css_selector('#div_split_history > table:nth-child(3) > tbody:nth-child(3)') # For Firefox
        splits = driver.find_element_by_css_selector('#splittable > tbody') # For Chrome
        if splits.text:
#             splits_2 = [x.split(' ') for x in splits.text.split('\n')]
#             for sp in splits_2:
#                 result2.append([str(i)] + sp)
            result2 += [[TICKER]+x.split(' ') for x in splits.text.split('\n')]
    except:
        error2.append(i)
driver.quit()
print('Time taken: ',time.time()-start_time)
data2 = pd.DataFrame(result2, columns=['code','split_date','ratio'])
print(data2.shape)
data2.to_csv(out_dir+'klse_morningstar_merge_split.csv', index=False)
display(data2.head(1))
assert(len(error2) == 0)

100%|███████████████████████████████████████████████████████████████████████████| 789/789 [1:27:22<00:00,  6.64s/it]


Time taken:  5253.496532440186
(181, 3)


,code,split_date,ratio
0,7086,11/10/2006,1:5


### Stage 3: Get past 10 years data

In [ ]:
# last time 5293 dont have record, check again
# balance = balance[~balance.code.isin(['5293'])]
# balance.shape

In [4]:
# Get the last 10 years of financial data from klse.i3investor.com
start_time = time.time()
options = webdriver.ChromeOptions()
options.add_argument("--incognito")
driver = webdriver.Chrome(chromedriver, options=options)
driver.switch_to.window(driver.current_window_handle)
driver.maximize_window()
result3_1, error3_1 = [], []
for i in tqdm(data1.code):
    try:
        driver.get('https://klse.i3investor.com/servlets/stk/fin/'+str(i)+'.jsp?type=last10fy')
#         cat = driver.find_element_by_class_name('boarAndSector').get_attribute('innerText').split(' : ')[-1]
        x = driver.find_elements_by_tag_name('table')[13].find_elements_by_tag_name('tr')
        if len(x) != 119:
            x = driver.find_elements_by_tag_name('table')[14].find_elements_by_tag_name('tr')
#         price_temp = 
#         price      = [str(i), cat, 'Price'] + [price_temp[1]] + price_temp[3:-1]
        price      = [str(i), 'Price'] + [x.get_attribute('innerText') for x in x[84].find_elements_by_tag_name('td')[3:-1]]
        result3_1.append(price)
        time.sleep(2)
    except Exception as e:
        error3_1.append([i, type(e).__name__])
driver.quit()
print('Time taken: ',time.time()-start_time)
data3_1 = pd.DataFrame(result3_1)
data3_1.columns = ['code','detail','Y1','Y2','Y3','Y4','Y5','Y6','Y7','Y8','Y9','Y10']
data3_1['code'] = data3_1['code'].astype(str)
data3_1.to_csv(out_dir+'klse_investor_10yr_price.csv', index=False)
print(data3_1.shape)
data3_1.head(1)

100%|███████████████████████████████████████████████████████████████████████████| 789/789 [1:18:14<00:00,  5.95s/it]


Time taken:  4706.48424911499
(787, 12)


,code,detail,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10
0,5250,Price,1.4300,1.5000,1.5400,1.4200,1.5400,1.5000,0.0000,None,None,None


In [5]:
def stage_3_morningstar(data1):
    # Get the last 10 years of financial data from financials.morningstar.com
    start_time = time.time()
    options = webdriver.ChromeOptions()
    options.add_argument("--incognito")
    driver = webdriver.Chrome(chromedriver, options=options)
    driver.switch_to.window(driver.current_window_handle)
    driver.maximize_window()
    data3_2, error3_2 = pd.DataFrame(), []

    driver.get('https://financials.morningstar.com/ratios/r.html?t=5398&culture=en&platform=sal&region=mys')
    time.sleep(2)
    driver.find_element_by_xpath('//*[@id="orderType"]').click()
    driver.find_element_by_xpath('//*[@id="Li2"]/ul/li[2]/a').click()

    for i in tqdm(data1.code):
        try:
            driver.get('https://financials.morningstar.com/ratios/r.html?t='+str(i)+'&culture=en&platform=sal&region=mys')
            time.sleep(2)
            year_end = [str(i), 'Year End'] + driver.find_element_by_id('financials').find_element_by_tag_name('thead')\
                                                    .get_attribute('innerText').split('\n')[0].split('\t')[2:]
            assert len(year_end) == 12, 'Number of year mismatch'

            table_1 = driver.find_element_by_id('financials').find_element_by_tag_name('tbody')\
                            .find_elements_by_tag_name('tr')[1::2]

            revenue    = [str(i), 'Revenue'] + table_1[0].get_attribute('innerText').split('\t')[2:]
            net_profit = [str(i), 'NP'] + table_1[4].get_attribute('innerText').split('\t')[2:]
            eps        = [str(i), 'EPS'] + table_1[5].get_attribute('innerText').split('\t')[2:]
            dps        = [str(i), 'DPS'] + table_1[6].get_attribute('innerText').split('\t')[2:]
            div_payout = [str(i), 'Div Payout'] + table_1[7].get_attribute('innerText').split('\t')[2:]
            shares     = [str(i), 'Shares'] + table_1[8].get_attribute('innerText').split('\t')[2:]
            nta        = [str(i), 'NTA'] + table_1[9].get_attribute('innerText').split('\t')[2:]
            ocf        = [str(i), 'OCF'] + table_1[10].get_attribute('innerText').split('\t')[2:]

            tabs = driver.find_element_by_class_name('in_tabs').find_elements_by_tag_name('li')

            tabs[0].click()
            table_2 = driver.find_element_by_id('tab-profitability').find_elements_by_tag_name('table')[-1]\
                            .find_element_by_tag_name('tbody').find_elements_by_tag_name('tr')[1::2]

            np_margin  = [str(i), 'NP Margin'] + table_2[1].get_attribute('innerText').split('\t')[2:]
            roe        = [str(i), 'ROE'] + table_2[5].get_attribute('innerText').split('\t')[2:]
    #         roic = [str(i), 'ROIC'] + table_2[6].get_attribute('innerText').split('\t')[2:]

            tabs[1].click()
            table_3 = driver.find_element_by_id('tab-growth').find_element_by_tag_name('table')\
                            .find_element_by_tag_name('tbody').find_elements_by_tag_name('tr')[1::2]

            rev_gr_yoy     = [str(i), 'Revenue GR'] + table_3[1].get_attribute('innerText').split('\t')[2:]
            rev_gr_5y_avg  = [str(i), 'Revenue GR 5Y avg'] + table_3[3].get_attribute('innerText').split('\t')[2:]
            rev_gr_10y_avg = [str(i), 'Revenue GR 10Y avg'] + table_3[4].get_attribute('innerText').split('\t')[2:]

            eps_gr_yoy     = [str(i), 'EPS GR'] + table_3[-4].get_attribute('innerText').split('\t')[2:]
            eps_gr_5y_avg  = [str(i), 'EPS GR 5Y avg'] + table_3[-2].get_attribute('innerText').split('\t')[2:]
            eps_gr_10y_avg = [str(i), 'EPS GR 10Y avg'] + table_3[-1].get_attribute('innerText').split('\t')[2:]

            tabs[2].click()
            table_4 = driver.find_element_by_id('tab-cashflow').find_element_by_tag_name('table')\
                            .find_element_by_tag_name('tbody').find_elements_by_tag_name('tr')[1::2]

            ocf_gr_yoy     = [str(i), 'OCF GR'] + table_4[0].get_attribute('innerText').split('\t')[2:]

            tabs[3].click()
            table_5 = driver.find_element_by_id('tab-financial').find_elements_by_tag_name('table')[-1]\
                            .find_element_by_tag_name('tbody').find_elements_by_tag_name('tr')[1::2]

            de_ratio = [str(i), 'D/E'] + table_5[-1].get_attribute('innerText').split('\t')[2:]

            data3_2 = pd.concat([data3_2, pd.DataFrame([year_end, revenue, net_profit, eps, dps, div_payout, shares, nta, ocf,
                                          np_margin, roe,# roic,
                                          rev_gr_yoy, rev_gr_5y_avg, rev_gr_10y_avg, eps_gr_yoy, eps_gr_5y_avg, eps_gr_10y_avg,
                                          ocf_gr_yoy,
                                          de_ratio])])
        except Exception as e:
            error3_2.append([i, type(e).__name__])
    driver.quit()
    print('Time taken: ',time.time()-start_time)
    return data3_2, error3_2

data3_2, error3_2 = stage_3_morningstar(data1)
data3_2.columns = ['code','detail','Y1','Y2','Y3','Y4','Y5','Y6','Y7','Y8','Y9','Y10']
print(data3_2.shape)
data3_2.to_csv(out_dir+'klse_morningstar_10yr.csv', index=False)
data3_2.head(1)

100%|█████████████████████████████████████████████████████████████████████████████| 789/789 [47:04<00:00,  3.58s/it]


Time taken:  2838.667927980423
(14839, 12)


,code,detail,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10
0,5250,Year End,2018-12,2017-12,2016-12,2015-12,2014-12,2013-12,2012-12,2011-12,2010-12,2009-12


In [16]:
# data3_3, error3_3 = stage_3_morningstar(balance)
# data3_3.columns = ['code','detail','Y1','Y2','Y3','Y4','Y5','Y6','Y7','Y8','Y9','Y10']
# print(data3_3.shape)
# temp = pd.concat([data3_2, data3_3])
# print(temp.code.nunique())
# temp.to_csv(out_dir+'klse_morningstar_10yr.csv', index=False)

100%|█████████████████████████████████████████████████████████████████████████████████| 8/8 [00:23<00:00,  2.96s/it]


Time taken:  38.47571063041687
(152, 12)


In [19]:
data3 = pd.concat([data3_1.fillna('—'), data3_2, data3_3]).sort_values(['code','detail'])
data3['code'] = data3['code'].astype(str).apply(lambda x: x.zfill(4))
print(data3.shape)
data3.to_csv(out_dir + 'klse_10yr_data.csv', index=False)
data3.head(1)

(15778, 12)


,code,detail,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10
18,0002,D/E,0.18,0.30,0.40,0.51,0.62,0.71,0.84,0.90,0.87,0.83


In [20]:
data3.code.nunique()

789

### Stage 4: Process each stock

In [29]:
error4, error4a, result4, symbol= [], [], [], ['—','\xa0-\xa0']
for i in tqdm(data3.code.unique()):
    stock = data3[data3.code == str(i).zfill(4)]
    net_profit = [float(x.replace(',','')) for x in stock[stock.detail=='NP'].values[0][2:] if x not in symbol]
    nYear = len(net_profit)
    try:
        assert(nYear >= 5)
        try:
            for y in [float(x.replace(',','')) for x in stock[stock.detail=='NP'].values[0][2:] if x not in symbol]: 
                assert(y >= 0)
            try:
                for y in [float(x.replace(',','')) for x in stock[stock.detail=='OCF'].values[0][2:] if x not in symbol]: 
                    assert(y >= 0)
#                 try:
#                     for y in [float(x.replace(',','')) for x in stock[stock.detail=='EPS GR'].values[0][2:] if x not in symbol]: 
#                         assert(y >= 0)
                try:
                    net_profit     = [int(x.replace(',','')) if x not in symbol else 0 for x in stock[stock.detail=='NP'].values[0][2:2+nYear] ]
                    nosh           = [int(x.replace(',','')) if x not in symbol else 0 for x in stock[stock.detail=='Shares'].values[0][2:2+nYear]]
                    price          = [float(str(x).replace(',','')) if x not in symbol else 0.0 for x in stock[stock.detail=='Price'].values[0][2:2+nYear]]
                    temp           = [np.around(net_profit[x] / (nosh[x] * price[x]) * 100, 2) for x in range(len(net_profit)) if price[x] != 0]
                    roi            = np.around(temp[0], 2)
                    roi_avg        = np.around(np.mean(temp), 2)
                    roi_std        = np.around(np.std(temp), 2)

                    rev_gr         = [float(x.replace(',','')) for x in stock[stock.detail=='Revenue GR'].values[0][2:] if x not in symbol]
                    rev_gr_avg     = np.around(np.mean(rev_gr), 2)
                    rev_gr_std     = np.around(np.std(rev_gr), 2)
                    rev_gr_5y_avg  = [float(x.replace(',','')) for x in stock[stock.detail=='Revenue GR 5Y avg'].values[0][2:] if x not in symbol]
                    rev_gr_10y_avg = [float(x.replace(',','')) for x in stock[stock.detail=='Revenue GR 10Y avg'].values[0][2:] if x not in symbol]
                    if len(rev_gr_5y_avg) == 0:
                        rev_gr_5y_avg = [0.0]
                    if len(rev_gr_10y_avg) == 0:
                        rev_gr_10y_avg = [0.0] 

                    np_margin      = [float(x.replace(',','')) for x in stock[stock.detail=='NP Margin'].values[0][2:] if x not in symbol]
                    np_margin_avg  = np.around(np.mean(np_margin), 2)
                    np_margin_std  = np.around(np.std(np_margin), 2)

                    eps            = [float(x.replace(',','')) if x not in symbol else 0.0 for x in stock[stock.detail=='EPS'].values[0][2:2+nYear]]
                    eps_avg        = np.around(np.mean(eps), 2)
                    eps_std        = np.around(np.std(eps), 2)      

                    eps_gr         = [float(x.replace(',','')) for x in stock[stock.detail=='EPS GR'].values[0][2:] if x not in symbol]
                    eps_gr_avg     = np.around(np.mean(eps_gr), 2)
                    eps_gr_std     = np.around(np.std(eps_gr), 2)
                    eps_gr_5y_avg  = [float(x.replace(',','')) for x in stock[stock.detail=='EPS GR 5Y avg'].values[0][2:] if x not in symbol]
                    eps_gr_10y_avg = [float(x.replace(',','')) for x in stock[stock.detail=='EPS GR 10Y avg'].values[0][2:] if x not in symbol]
                    if len(eps_gr_5y_avg) == 0:
                        eps_gr_5y_avg = [0.0]
                    if len(eps_gr_10y_avg) == 0:
                        eps_gr_10y_avg = [0.0]

                    ey             = [np.around(eps[x]/price[x]*100, 2) for x in range(len(price)) if price[x] != 0]# EPS / Price
                    ey_avg         = np.around(np.mean(ey),2)
                    ey_std         = np.around(np.std(ey),2)         

                    pe             = [np.around(price[x]/eps[x], 2) for x in range(len(price)) if eps[x] != 0]# Price / EPS
                    pe_mid         = np.around((max(pe) + min(pe))/2, 2)
                    pe_avg         = np.around(np.mean(pe), 2)
                    pe_std         = np.around(np.std(pe), 2)

                    dps            = [float(x.replace(',','')) if x not in symbol else 0.0 for x in stock[stock.detail=='DPS'].values[0][2:2+nYear]]
                    if len(dps) == 0:
                        dps = [0.0]
                    dps_avg        = np.around(np.mean(dps), 2)
                    dps_std        = np.around(np.std(dps), 2)

                    dpayout        = [float(x.replace(',','')) for x in stock[stock.detail=='Div Payout'].values[0][2:] if x not in symbol]
                    if len(dpayout) == 0:
                        dpayout = [0.0]
                    dpayout_avg    = np.around(np.mean(dpayout), 2)
                    dpayout_std    = np.around(np.std(dpayout), 2)

                    dy             = [np.around(dps[x]/price[x]*100, 2) for x in range(len(price)) if price[x] != 0]# DPS / Price
                    dy_avg         = np.around(np.mean(dy),2)
                    dy_std         = np.around(np.std(dy),2)

                    roe_gr         = [float(x.replace(',','')) for x in stock[stock.detail=='ROE'].values[0][2:] if x not in symbol]
                    roe_avg        = np.around(np.mean(roe_gr), 2)
                    roe_std        = np.around(np.std(roe_gr), 2)            

                    de             = [float(x.replace(',','')) for x in stock[stock.detail=='D/E'].values[0][2:] if x not in symbol]
                    de_avg         = np.around(np.mean(de), 2)
                    de_std         = np.around(np.std(de), 2)  

                    ocf_gr         = [float(x.replace(',','')) for x in stock[stock.detail=='OCF GR'].values[0][2:] if x not in symbol]
                    ocf_gr_avg     = np.around(np.mean(ocf_gr), 2)
                    ocf_gr_std     = np.around(np.std(ocf_gr), 2)

                    nMerge         = 0
                    nSplit         = 0
                    dates          = [datetime.strptime(x, '%Y-%m') for x in stock[stock.detail=='Year End'].values[0][2:] if x not in symbol]
                    last_date      = dates[-1]
                    annual         = dates[0]
                    temp1          = data2[data2.code == i]   
                    try:
                        temp1['split_date'] = temp1.split_date.apply(lambda x: datetime.strptime(x, '%m/%d/%Y'))
                    except:
                        temp1          = temp1[temp1.split_date >= last_date]
                    for n, row in temp1.iterrows():
                        split_ratio = row.ratio.split(':')
                        if len(split_ratio) == 2:
                            if split_ratio[0] < split_ratio[-1]: nMerge += 1   
                            else: nSplit += 1
                        else:
                            error4a.append([str(i).zfill(4), 'Split date format error'])
                    result4.append([str(i).zfill(4), annual, nYear, nMerge, nSplit,
                                    roi, roi_avg, roi_std, np_margin_avg, np_margin_std, 
                                    eps_avg, eps_std, eps_gr_avg, eps_gr_std, eps_gr_5y_avg[0], eps_gr_10y_avg[0], 
                                    ey_avg, ey_std, roe_avg, roe_std, pe_mid, pe_avg, pe_std, 
                                    dps_avg, dps_std, dpayout_avg, dpayout_std, dy_avg, dy_std, 
                                    de_avg, de_std, ocf_gr_avg, ocf_gr_std,
                                    rev_gr_avg, rev_gr_std, rev_gr_5y_avg[0], rev_gr_10y_avg[0]])
                except:
                    error4.append([str(i).zfill(4), 'Calculation Error'])
#                 except:
#                     error4.append([str(i).zfill(4), 'Negative EPS GR'])
            except:
                error4.append([str(i).zfill(4), 'Negative Operating Cash Flow']) 
        except:
            error4.append([str(i).zfill(4), 'Negative Net Profit'])
    except:
        error4.append([str(i).zfill(4), 'Less than 5 years'])

100%|████████████████████████████████████████████████████████████████████████████| 789/789 [00:01<00:00, 464.56it/s]


In [13]:
er_df_4 = pd.DataFrame(error4, columns=['code','reason'])
display(er_df_4.reason.value_counts())
if len(error4a) != 0: 
    print(len(error4a))

data4 = pd.DataFrame(result4, columns=['code','Annual','nYear','nMerge','nSplit',
                                       'ROI','ROI avg','ROI std','NPM avg','NPM std',
                                       'EPS avg','EPS std','EPS GR avg','EPS GR std','EPS GR 5Y avg','EPS GR 10Y avg',
                                       'EY avg','EY std','ROE avg','ROE std','P/E mid','P/E avg','P/E std',
                                       'DPS avg','DPS std','DPayout avg','DPayout std','DY avg','DY std',
                                       'D/E avg','D/E std','OCF GR avg','OCF GR std',
                                       'Revenue GR avg','Revenue GR std','Revenue GR 5Y avg','Revenue GR 10Y avg'])
# data4 = data4[data4['ROI avg'] >= 8]
# data4 = data4[data4['EY avg'] >= 8]
# data4 = data4[data4['ROE avg'] >= 10]
# data4 = data4[data4['EPS GR avg'] >= 8]
# data4 = data4[data4['D/E avg'] <= 0.5]
# data4 = data4[data4['P/E'] < data4['P/E mid']]
# data4 = data4[(data4['P/E'] >= 6.66) & (data['P/E'] <= 12.5)]
data4 = data4.merge(stock_code,'left')
data4 = data4.merge(data1,'left')
temp = data4.columns.tolist()
data4 = data4[temp[:1]+temp[-13:]+temp[1:-13]]
print(data4.shape)
data4.sort_values(['category','code'],ascending=False,inplace=True)
data4.reset_index(drop=True,inplace=True)
# data4.to_csv(out_dir+'klse_stock_stage_4.csv',index=False)
display(data4.head(1))

Negative Net Profit    424
Less than 5 years       18
Name: reason, dtype: int64

(347, 50)


,code,name,category,price,NTA,52w low,52w high,market cap,ROE,P/E,...,DY avg,DY std,D/E avg,D/E std,Operating CF GR avg,Operating CF GR std,Revenue GR avg,Revenue GR std,Revenue GR 5Y avg,Revenue GR 10Y avg
0,8524,TALIWORKS CORPORATION BERHAD,Utilities,0.75,51.25,0.635,1.03,"1,511.9M",0.07,20.0,...,2.1,2.33,0.46,0.21,76.91,99.6,11.58,21.04,4.91,4.69


In [28]:
# With Negative EPS GR filter

Negative Net Profit    424
Negative EPS GR        338
Less than 5 years       18
Name: reason, dtype: int64

(9, 50)


,code,name,category,price,NTA,52w low,52w high,market cap,ROE,P/E,...,DY avg,DY std,D/E avg,D/E std,OCF GR avg,OCF GR std,Revenue GR avg,Revenue GR std,Revenue GR 5Y avg,Revenue GR 10Y avg
0,0138,MY E.G. SERVICES BERHAD,Technology,0.975,0.1981,0.765,1.75,"3,516.1M",34.43,14.0,...,0.25,0.4,0.09,0.06,61.44,152.58,30.55,29.26,40.87,30.72


In [15]:
# With Negative Operating Cash Flow filter

Negative Net Profit             424
Negative Operating Cash Flow    158
Less than 5 years                18
Name: reason, dtype: int64

(189, 50)


,code,name,category,price,NTA,52w low,52w high,market cap,ROE,P/E,...,DY avg,DY std,D/E avg,D/E std,Operating CF GR avg,Operating CF GR std,Revenue GR avg,Revenue GR std,Revenue GR 5Y avg,Revenue GR 10Y avg
0,6742,YTL POWER INTERNATIONAL BHD,Utilities,0.655,1.59,0.48,0.915,"5,343.6M",3.08,13.0,...,4.48,2.3,1.9,0.31,16.32,70.5,11.78,39.25,-4.06,7.08


In [30]:
# With Negative EPS GR & Operating Cash Flow filter

Negative Net Profit             424
Negative EPS GR                 183
Negative Operating Cash Flow    158
Less than 5 years                18
Name: reason, dtype: int64

(6, 50)


,code,name,category,price,NTA,52w low,52w high,market cap,ROE,P/E,...,DY avg,DY std,D/E avg,D/E std,OCF GR avg,OCF GR std,Revenue GR avg,Revenue GR std,Revenue GR 5Y avg,Revenue GR 10Y avg
0,0138,MY E.G. SERVICES BERHAD,Technology,0.975,0.1981,0.765,1.75,"3,516.1M",34.43,14.0,...,0.25,0.4,0.09,0.06,61.44,152.58,30.55,29.26,40.87,30.72


In [34]:
data4

,code,name,category,price,NTA,52w low,52w high,market cap,ROE,P/E,...,DY avg,DY std,D/E avg,D/E std,OCF GR avg,OCF GR std,Revenue GR avg,Revenue GR std,Revenue GR 5Y avg,Revenue GR 10Y avg
0,0138,MY E.G. SERVICES BERHAD,Technology,0.975,0.1981,0.765,1.75,"3,516.1M",34.43,14.0,...,0.25,0.40,0.09,0.06,61.44,152.58,30.55,29.26,40.87,30.72
1,4731,SCIENTEX BERHAD,Industrial Products & Services,7.480,4.5700,5.960,9.87,"3,858.8M",16.32,10.0,...,2.01,0.60,0.12,0.08,27.26,35.10,20.80,10.69,15.34,20.34
2,7277,DIALOG GROUP BERHAD,Energy,3.090,0.7030,2.700,3.57,"17,432.7M",15.31,29.0,...,0.85,0.31,0.31,0.13,30.26,84.70,9.76,19.41,-1.33,8.01
3,7084,QL RESOURCES BERHAD,Consumer Products & Services,7.900,1.2700,6.500,8.65,"12,817.3M",11.63,53.0,...,0.70,0.16,0.31,0.10,12.99,45.58,10.56,4.33,8.05,9.98
4,5275,MYNEWS HOLDINGS BERHAD,Consumer Products & Services,0.705,0.4500,0.575,1.49,480.9M,7.47,21.0,...,0.61,0.13,0.09,0.08,223.65,324.39,24.27,4.87,0.00,0.00
5,0186,PERAK TRANSIT BERHAD,Consumer Products & Services,0.180,0.2324,0.125,0.27,256.1M,12.01,6.0,...,2.13,2.13,0.73,0.23,34.52,56.12,12.48,10.08,12.01,0.00


In [16]:
data4.columns

Index(['code', 'name', 'category', 'price', 'NTA', '52w low', '52w high',
       'market cap', 'ROE', 'P/E', 'EPS', 'EY', 'DPS', 'DY', 'Annual', 'nYear',
       'nMerge', 'nSplit', 'ROI', 'ROI avg', 'ROI std', 'NPM avg', 'NPM std',
       'EPS avg', 'EPS std', 'EPS GR avg', 'EPS GR std', 'EPS GR 5Y avg',
       'EPS GR 10Y avg', 'EY avg', 'EY std', 'ROE avg', 'ROE std', 'P/E mid',
       'P/E avg', 'P/E std', 'DPS avg', 'DPS std', 'DPayout avg',
       'DPayout std', 'DY avg', 'DY std', 'D/E avg', 'D/E std',
       'Operating CF GR avg', 'Operating CF GR std', 'Revenue GR avg',
       'Revenue GR std', 'Revenue GR 5Y avg', 'Revenue GR 10Y avg'],
      dtype='object')

In [17]:
data4[data4.Price > data4.NTA]

,code,name,category,price,NTA,52w low,52w high,market cap,ROE,P/E,...,DY avg,DY std,D/E avg,D/E std,Operating CF GR avg,Operating CF GR std,Revenue GR avg,Revenue GR std,Revenue GR 5Y avg,Revenue GR 10Y avg
1,6033,PETRONAS GAS BERHAD,Utilities,15.300,6.6939,13.360,18.388,"30,274.6M",14.61,16.0,...,3.25,0.80,0.15,0.08,9.32,22.03,5.57,13.72,4.44,5.55
2,5347,TENAGA NASIONAL BHD,Utilities,12.100,10.1957,10.365,13.704,"68,811.4M",7.81,15.0,...,2.12,0.93,0.70,0.09,9.22,25.07,6.32,4.14,5.89,6.71
5,3069,MEGA FIRST CORPORATION BERHAD,Utilities,4.590,3.7800,3.100,5.600,"2,110.6M",8.84,14.0,...,2.73,1.02,0.06,0.11,-6.04,24.65,8.58,18.57,6.61,5.63
8,6645,LINGKARAN TRANS KOTA HOLDINGS BERHAD,Transportation & Logistics,3.770,2.0700,3.300,5.000,"2,003.2M",24.23,8.0,...,4.76,0.64,2.43,0.97,6.83,14.19,6.16,9.03,6.65,5.66
11,5246,WESTPORTS HOLDINGS BERHAD,Transportation & Logistics,3.500,0.7508,2.970,4.540,"11,935.0M",23.08,20.0,...,2.69,1.14,0.61,0.20,12.15,28.23,7.33,7.48,3.52,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,1562,BERJAYA SPORTS TOTO BERHAD,Consumer Products & Services,2.220,0.5900,1.940,2.820,"2,999.3M",19.78,19.0,...,6.20,0.99,0.72,0.22,-3.82,14.29,5.23,9.12,9.30,5.62
180,0049,OCEANCASH PACIFIC BERHAD,Consumer Products & Services,0.415,0.4015,0.290,0.820,101.8M,7.22,14.0,...,1.04,1.07,0.07,0.04,15.80,51.26,5.19,5.57,4.43,5.56
181,9598,PINTARAS JAYA BHD,Construction,2.490,1.9800,1.800,3.430,413.0M,15.43,8.0,...,4.68,1.90,0.06,0.00,16.05,62.13,25.56,75.19,9.36,9.26
183,5263,SUNWAY CONSTRUCTION GROUP BERHAD,Construction,1.620,0.4800,1.250,2.200,"2,094.5M",20.83,16.0,...,2.19,1.87,0.03,0.00,54.10,88.77,8.21,10.89,4.17,0.00


### Stage 5: Intrinsic Value & Fair Value

In [18]:
result5 = []
for n, x in tqdm(data4.iterrows(), total=len(data4)):
    price_10y = [np.around(x['Price'] * ((1 + (x['EPS GR avg'] / 100))**i) ,2) for i in range(5)]
    total_eps = np.around(sum(price_10y), 2)
    expected_price = np.around(price_10y[-1] * x['P/E mid'],2)
    total_dvd = np.around(total_eps * (x['DPayout avg'] / 100), 2)
    total_return = np.around(total_eps + total_dvd, 2)
    intrinsic = np.around(total_return / ((1 + (x['EPS GR avg'] / 100))**5), 2)
    price_A = np.around(intrinsic * 0.75, 2)
    price_B = np.around((x['52w high'] - x['52w low']) * 0.33 + x['52w low'], 2)
    result5.append([str(x['code']), intrinsic, price_A, price_B])

data5 = data4.merge(pd.DataFrame(result5, columns=['code','intrinsic','Price_A','Price_B']))
print(data5.shape)
data5.to_csv(out_dir+'klse_stock_intrinsic.csv',index=False)
data5.head(1)

100%|███████████████████████████████████████████████████████████████████████████| 189/189 [00:00<00:00, 3373.99it/s]


(189, 53)


,code,name,category,price,NTA,52w low,52w high,market cap,ROE,P/E,...,D/E std,Operating CF GR avg,Operating CF GR std,Revenue GR avg,Revenue GR std,Revenue GR 5Y avg,Revenue GR 10Y avg,intrinsic,price_A,price_B
0,6742,YTL POWER INTERNATIONAL BHD,Utilities,0.655,1.59,0.48,0.915,"5,343.6M",3.08,13.0,...,0.31,16.32,70.5,11.78,39.25,-4.06,7.08,5.38,4.04,0.62


In [19]:
# Stocks which have prices lower than fair value price B
data5[data5.Price < data5.Price_B]

,code,name,category,price,NTA,52w low,52w high,market cap,ROE,P/E,...,D/E std,Operating CF GR avg,Operating CF GR std,Revenue GR avg,Revenue GR std,Revenue GR 5Y avg,Revenue GR 10Y avg,intrinsic,price_A,price_B
4,4677,YTL CORPORATION BERHAD,Utilities,0.790,1.2400,0.600,1.250,"8,708.0M",0.89,72.0,...,0.32,13.39,45.73,9.42,26.71,-1.89,6.93,9.27,6.95,0.81
6,7210,FREIGHT MANAGEMENT HOLDINGS BERHAD,Transportation & Logistics,0.450,1.0400,0.380,0.700,125.7M,4.40,10.0,...,0.04,17.71,62.59,9.14,4.60,6.22,9.04,3.58,2.68,0.49
7,7117,CJ CENTURY LOGISTICS HOLDINGS BERHAD,Transportation & Logistics,0.245,0.8100,0.205,0.470,96.6M,-2.36,-13.0,...,0.09,5.10,45.01,8.22,13.82,9.41,9.36,2.46,1.84,0.29
8,6645,LINGKARAN TRANS KOTA HOLDINGS BERHAD,Transportation & Logistics,3.770,2.0700,3.300,5.000,"2,003.2M",24.23,8.0,...,0.97,6.83,14.19,6.16,9.03,6.65,5.66,23.13,17.35,3.86
9,6521,SURIA CAPITAL HOLDINGS BERHAD,Transportation & Logistics,0.880,3.2196,0.820,1.450,304.3M,4.69,6.0,...,0.13,16.61,52.42,10.22,31.32,8.69,3.42,4.50,3.38,1.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,0049,OCEANCASH PACIFIC BERHAD,Consumer Products & Services,0.415,0.4015,0.290,0.820,101.8M,7.22,14.0,...,0.04,15.80,51.26,5.19,5.57,4.43,5.56,1.50,1.12,0.46
182,8761,BREM HOLDING BERHAD,Construction,0.565,1.7000,0.520,0.800,195.2M,6.01,6.0,...,0.04,38.94,102.60,8.45,35.31,-3.32,0.58,0.96,0.72,0.61
185,4723,JAKS RESOURCES BERHAD,Construction,0.885,1.5200,0.620,1.570,576.2M,10.97,5.0,...,0.25,68.14,197.62,11.12,20.55,2.34,10.73,NaN,NaN,0.93
187,0192,INTA BINA GROUP BERHAD,Construction,0.180,0.2560,0.150,0.325,96.3M,16.37,4.0,...,0.10,54.56,130.02,12.53,13.55,0.00,0.00,NaN,NaN,0.21


In [20]:
# Stocks which have prices lower than fair value price B and average NPM of 10% or higher
data5[(data5.Price < data5.Price_B) & (data5['NPM avg'] >= 10)]

,code,name,category,price,NTA,52w low,52w high,market cap,ROE,P/E,...,D/E std,Operating CF GR avg,Operating CF GR std,Revenue GR avg,Revenue GR std,Revenue GR 5Y avg,Revenue GR 10Y avg,intrinsic,price_A,price_B
8,6645,LINGKARAN TRANS KOTA HOLDINGS BERHAD,Transportation & Logistics,3.770,2.0700,3.300,5.000,"2,003.2M",24.23,8.0,...,0.97,6.83,14.19,6.16,9.03,6.65,5.66,23.13,17.35,3.86
9,6521,SURIA CAPITAL HOLDINGS BERHAD,Transportation & Logistics,0.880,3.2196,0.820,1.450,304.3M,4.69,6.0,...,0.13,16.61,52.42,10.22,31.32,8.69,3.42,4.50,3.38,1.03
10,5267,XIN HWA HOLDINGS BERHAD,Transportation & Logistics,0.400,0.8700,0.245,0.840,86.4M,1.86,25.0,...,0.06,-5.73,10.52,-0.44,5.13,1.17,0.00,NaN,NaN,0.44
15,2194,MMC CORPORATION BERHAD,Transportation & Logistics,0.620,3.0400,0.450,1.250,"1,887.9M",2.76,7.0,...,0.99,-0.11,41.62,0.50,29.50,-7.71,-5.25,1.64,1.23,0.71
16,0078,GD EXPRESS CARRIER BERHAD,Transportation & Logistics,0.195,0.0900,0.115,0.360,"1,100.1M",5.11,42.0,...,0.13,15.66,46.91,15.50,5.48,14.61,15.38,0.57,0.43,0.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,3026,DUTCH LADY MILK INDUSTRIES BERHAD,Consumer Products & Services,43.500,2.2600,34.000,64.500,"2,784.0M",71.18,27.0,...,NaN,28.81,76.58,4.88,5.69,1.31,3.95,228.56,171.42,44.06
174,2836,CARLSBERG BREWERY MALAYSIA BERHAD,Consumer Products & Services,24.400,0.5100,17.380,39.260,"7,460.3M",186.65,26.0,...,NaN,20.17,37.00,7.72,9.09,4.97,7.52,97.45,73.09,24.60
177,0197,WEGMANS HOLDINGS BERHAD,Consumer Products & Services,0.130,0.1600,0.100,0.370,65.0M,9.94,8.0,...,0.17,59.76,97.78,27.04,25.80,0.00,0.00,0.15,0.11,0.19
179,0136,GREENYIELD BERHAD,Consumer Products & Services,0.085,0.1620,0.070,0.165,28.4M,6.23,8.0,...,0.12,1.71,47.55,7.17,22.30,-2.30,2.37,1.00,0.75,0.10


In practical, intrinsic value and price A are way beyond from the current price because there is no consistency of positive growth rate for any stock, my suggestion will be excluding the idea of positive growth rate, but filter stocks with positive earnings or net profit margin at 5% or 10%  
 <br />
Another method would be filtering stocks according to the dividend yield over the past 10 years. Although this method is not recommended by the author,but it returns more stock options and most of them has dividend yield of 10% or higher

### Process by Dividend Yield

In [21]:
data6 = pd.DataFrame(result4, columns=['code','Annual','nYear','nMerge','nSplit',
                                       'ROI','ROI avg','ROI std','NPM avg','NPM std',
                                       'EPS avg','EPS std','EPS GR avg','EPS GR std','EPS GR 5Y avg','EPS GR 10Y avg',
                                       'EY avg','EY std','ROE avg','ROE std','P/E mid','P/E avg','P/E std',
                                       'DPS avg','DPS std','DPayout avg','DPayout std','DY avg','DY std',
                                       'D/E avg','D/E std','OCF GR avg','OCF GR std',
                                       'Revenue GR avg','Revenue GR std','Revenue GR 5Y avg','Revenue GR 10Y avg'])
data6 = data6.merge(stock_code,'left')
data6 = data6.merge(data1,'left')
temp = data6.columns.tolist()
data6 = data6[temp[:1]+temp[-13:]+temp[1:-13]]
print(data6.shape)
data6.sort_values('DY avg',ascending=False,inplace=True)
data6.reset_index(drop=True,inplace=True)
data6.to_csv(out_dir+'klse_stock_dividend_yield.csv',index=False)
data6.head(1)

(189, 50)


,code,name,category,price,NTA,52w low,52w high,market cap,ROE,P/E,...,DY avg,DY std,D/E avg,D/E std,Operating CF GR avg,Operating CF GR std,Revenue GR avg,Revenue GR std,Revenue GR 5Y avg,Revenue GR 10Y avg
0,5108,ICAPITAL.BIZ BERHAD,Closed-End Fund,2.04,3.18,1.85,2.47,285.6M,0.85,76.0,...,3196.84,2333.95,0.18,0.08,12.64,55.46,3.39,7.63,-0.82,3.1


In [22]:
data6.sort_values('DY avg',ascending=False)

,code,name,category,price,NTA,52w low,52w high,market cap,ROE,P/E,...,DY avg,DY std,D/E avg,D/E std,Operating CF GR avg,Operating CF GR std,Revenue GR avg,Revenue GR std,Revenue GR 5Y avg,Revenue GR 10Y avg
0,5108,ICAPITAL.BIZ BERHAD,Closed-End Fund,2.040,3.1800,1.850,2.470,285.6M,0.85,76.0,...,3196.84,2333.95,0.18,0.08,12.64,55.46,3.39,7.63,-0.82,3.10
1,7811,SAPURA INDUSTRIAL BERHAD,Industrial Products & Services,0.420,1.4600,0.200,0.815,30.6M,2.25,13.0,...,9.05,4.76,0.18,0.09,31.74,95.94,0.29,9.94,-3.56,-0.50
2,7202,CLASSIC SCENIC BERHAD,Consumer Products & Services,0.600,0.7700,0.570,1.090,72.3M,6.74,12.0,...,8.18,2.94,NaN,NaN,13.23,50.66,6.12,15.22,3.73,2.87
3,3859,MAGNUM BERHAD,Consumer Products & Services,2.010,1.7400,1.700,2.900,"2,889.9M",9.54,12.0,...,7.64,6.11,0.52,0.29,-5.16,22.46,-2.11,5.37,-2.00,-1.47
4,5121,HEKTAR REAL ESTATE INVESTMENT TRUST,Real Estate Investment Trusts,0.680,1.3192,0.550,1.160,314.1M,1.88,27.0,...,7.22,0.77,0.73,0.17,10.18,35.28,4.67,4.84,2.36,4.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,8117,POLY GLASS FIBRE (M) BERHAD,Industrial Products & Services,0.200,1.0541,0.200,0.475,32.0M,1.24,15.0,...,0.00,0.00,0.08,0.04,93.37,125.54,8.74,12.71,12.24,6.74
185,4723,JAKS RESOURCES BERHAD,Construction,0.885,1.5200,0.620,1.570,576.2M,10.97,5.0,...,0.00,0.00,0.40,0.25,68.14,197.62,11.12,20.55,2.34,10.73
186,0078,GD EXPRESS CARRIER BERHAD,Transportation & Logistics,0.195,0.0900,0.115,0.360,"1,100.1M",5.11,42.0,...,0.00,0.00,0.17,0.13,15.66,46.91,15.50,5.48,14.61,15.38
187,5289,TECHBOND GROUP BERHAD,Industrial Products & Services,0.540,0.5700,0.480,0.960,124.2M,6.75,14.0,...,0.00,0.00,0.02,0.00,37.43,75.08,6.74,9.08,0.00,0.00


 #### Thank you for reading, using, and exploring this, hope to see you soon